<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará productos de supermercado para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esos productos hayan utilizado las palabras en sus descripciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Se consumirán los datos de todos los productos del supermercado La Gallega.

In [2]:
dfProductos = pd.read_json('datosLaGallega.json', orient='records')
dfProductos.head()

,descripcion,precio,linkFoto
0,rocio vegetal mazola x 120 ml. manteca,363.35,https://www.lagallega.com.ar/Archivos/Articulo...
1,rocio vegetal natura x 120 ml.,363.64,https://www.lagallega.com.ar/Archivos/Articulo...
2,rocio vegetal mazola x 120 ml. original,376.65,https://www.lagallega.com.ar/Archivos/Articulo...
3,rocio vegetal de girasol lira x 120 gr.,382.70,https://www.lagallega.com.ar/Archivos/Articulo...
4,rocio vegetal fritolim x 120 ml. clasico,394.34,https://www.lagallega.com.ar/Archivos/Articulo...


In [3]:
print("Cantidad de documentos:", dfProductos.shape[0])

Cantidad de documentos: 4983


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in dfProductos[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

/var/folders/62/k956tb7121v1t52bjhsj036m0000gn/T/ipykernel_19633/3977940082.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [5]:
# Demos un vistazo
sentence_tokens[:2]

[['rocio', 'vegetal', 'mazola', 'x', '120', 'ml', 'manteca'],
 ['rocio', 'vegetal', 'natura', 'x', '120', 'ml']]

### 2 - Crear los vectores (word2vec)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [7]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [8]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [9]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 4983


In [10]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1125


### 3 - Entrenar embeddings

In [11]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 253677.65625
Loss after epoch 1: 176535.46875
Loss after epoch 2: 177144.875
Loss after epoch 3: 176819.125
Loss after epoch 4: 174855.8125
Loss after epoch 5: 159336.0625
Loss after epoch 6: 142742.125
Loss after epoch 7: 135442.75
Loss after epoch 8: 129364.625
Loss after epoch 9: 124245.5
Loss after epoch 10: 118633.875
Loss after epoch 11: 115493.75
Loss after epoch 12: 112345.75
Loss after epoch 13: 109497.375
Loss after epoch 14: 100404.5
Loss after epoch 15: 98001.0
Loss after epoch 16: 96630.25
Loss after epoch 17: 97436.5
Loss after epoch 18: 96133.25
Loss after epoch 19: 96220.5


(450470, 745760)

### 4 - Ensayar

In [12]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["chocolate"], topn=10)

[('aireado', 0.8248280882835388),
 ('block', 0.8090999722480774),
 ('55', 0.8061637282371521),
 ('milka', 0.7970662713050842),
 ('almendras', 0.7966142296791077),
 ('cofler', 0.7935138940811157),
 ('mousse', 0.7844582200050354),
 ('95', 0.7564383149147034),
 ('chocolinas', 0.7528076767921448),
 ('king', 0.7399789690971375)]

In [13]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["leche"], topn=10)

[('500', -0.0650559663772583),
 ('300', -0.1423555463552475),
 ('150', -0.18001262843608856),
 ('y', -0.18614210188388824),
 ('sed', -0.18663932383060455),
 ('gatillo', -0.18729367852210999),
 ('cif', -0.19191670417785645),
 ('bano', -0.1921139508485794),
 ('s', -0.19681377708911896),
 ('extra', -0.19989842176437378)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["harina"], topn=10)

[('maiz', 0.9302715063095093),
 ('pisingallo', 0.9117167592048645),
 ('pronta', 0.8983747363090515),
 ('trigo', 0.8978471159934998),
 ('porotos', 0.8954564332962036),
 ('lentejas', 0.8874412775039673),
 ('pallares', 0.8870183825492859),
 ('alubia', 0.8794804811477661),
 ('garbanzos', 0.8732545971870422),
 ('negros', 0.8682364225387573)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["alfajor"], topn=5)

[('mousse', 0.9049503207206726),
 ('terrabusi', 0.8885467052459717),
 ('mini', 0.8671366572380066),
 ('milka', 0.8499336838722229),
 ('almendras', 0.8470690846443176)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["yogur"], topn=5)

[('firme', 0.8876400589942932),
 ('batido', 0.8815719485282898),
 ('descremado', 0.8813347816467285),
 ('bebible', 0.8707025647163391),
 ('entero', 0.8696739673614502)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["yogur"])

[('para', -0.135587677359581),
 ('asurin', -0.14505426585674286),
 ('dea', -0.183661088347435),
 ('sed', -0.18676745891571045),
 ('cm', -0.19196347892284393),
 ('y', -0.19253818690776825),
 ('p', -0.2023717612028122),
 ('original', -0.2088073492050171),
 ('glade', -0.20925141870975494),
 ('paladini', -0.20931023359298706)]

In [20]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("yogur")
print(vector_love)

[ 0.18141448  0.03481099 -0.20967692  0.13575491 -0.40767136 -0.3105617
 -0.17472002  0.37860593 -0.14935356  0.01362702 -0.10638512  0.05919974
 -0.24458964 -0.02992211 -0.24574077  0.17964515 -0.22886842 -0.2537028
  0.30055156 -0.4811053   0.06293266 -0.13841753  0.14720058  0.33398667
 -0.3063417   0.23242737 -0.55310744 -0.21574664  0.3909131  -0.17514239
 -0.18098107  0.26541153 -0.23786853  0.13885847 -0.37808666  0.06891818
  0.04198392  0.08803821  0.0608006  -0.43512517  0.0059677  -0.43467504
 -0.46412292 -0.36093786  0.21336523  0.593584   -0.15563585  0.00097944
  0.23452303  0.07357759 -0.09213521  0.10834215  0.16870369  0.11546773
 -0.05881082  0.08069296 -0.6689662   0.17417893  0.15701976 -0.43581316
  0.41992712 -0.24421665  0.02909892  0.3304809  -0.36458725 -0.02123063
  0.03806241 -0.5598865  -0.29343647  0.09701548  0.22731157 -0.01835622
 -0.3089166  -0.5040655   0.38086754  0.17329977 -0.14408606  0.16661452
 -0.10724964  0.13841943 -0.13816491 -0.3911341   0.1

In [21]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('yogur', 1.0),
 ('firme', 0.8876399993896484),
 ('batido', 0.8815719485282898),
 ('descremado', 0.881334662437439),
 ('bebible', 0.8707025647163391),
 ('entero', 0.8696738481521606),
 ('yogs', 0.8569919466972351),
 ('gran', 0.850754976272583),
 ('colchon', 0.8389526009559631),
 ('sancor', 0.8190739154815674)]

### 5 - Visualizar agrupación de vectores

In [23]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [26]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
# fig.show(renderer="colab") # esto para plotly en colab
fig.show() # esto para plotly en colab

In [27]:
# Graficar los embeddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab
fig.show() # esto para plotly en colab

In [28]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Conclusiones

- En cuanto a los `chocolates` se agrupan en base a marcas o tamaños.
- En cuanto a la similitud negativa de `leche` y de `yogur` se puede observar que arroja resultados totalmente discordantes con la palabra buscada.
- En cuanto a la búsqueda de similitud positiva la `harina` arroja resultados diferentes tipos de granos o legumbres
- En cuanto a `yogur` sucede de manera similar que con `chocolates`


Para concluir, se puede observar en los gráficos tridimensionales de embeddings que para este dataset de descripciones de productos de supermercados se nota claramente la agrupación de productos similares como el vino y sus variedades, granos o legumbres, los lácteos, entre otros. El espacio vectorial de embeddings logra interceptar similitudes entre documentos de la misma clasificación del lenguaje cotidiano o categorizando como un supermercado lo haría.
